In [1]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 1.6 MB/

In [2]:
pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [8]:
pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.3 MB/s eta 0:00:00


In [9]:
pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 25.2 MB/s eta 0:00:00


In [3]:
import nltk
import spacy
from spacy import displacy
import numpy as np
import PyPDF2
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import string
import docx
import pymupdf
import gradio as gr
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [4]:
!python -m spacy download en_core_web_md
nlp = spacy.load("en_core_web_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 6.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [24]:
def read_document(doc_type, file_path):
    if doc_type == "PDF":
        return read_pdf(file_path)
    elif doc_type == "TXT":
        return read_txt(file_path)
    elif doc_type == "DOCX":
        return read_docx(file_path)
    else:
        raise ValueError("Unsupported document type.")

def read_pdf(file_path):
    text = ""
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in range(len(reader.pages)):
            text += reader.pages[page].extract_text()
    return text

def read_txt(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

def read_docx(file_path):
    doc = docx.Document(file_path)
    return "\n".join([paragraph.text for paragraph in doc.paragraphs])

def check_doc_similarity(doc_type, doc_1, doc_2):
    data_1 = read_document(doc_type, doc_1)
    data_2 = read_document(doc_type, doc_2)

    data_1 = preprocess_text(data_1)
    data_2 = preprocess_text(data_2)

    doc_1_nlp = nlp(data_1)
    doc_2_nlp = nlp(data_2)
    similarity = round(doc_1_nlp.similarity(doc_2_nlp), 2)

    return "Similarity score of documents: ", similarity

def catch_related_words(word):
    word_vector = nlp.vocab.vectors[nlp.vocab.strings[word]]
    most_similar_words = nlp.vocab.vectors.most_similar(np.asarray([word_vector]), n=50)
    similar_words_list = [nlp.vocab.strings[w] for w in most_similar_words[0][0]]
    return similar_words_list
'''
def check_categorical_similarity(category, doc_type, docs):
    documents = [read_document(doc_type, document) for document in docs]
    documents = [preprocess_text(document) for document in documents if document is not None]
    similarities = []
    for i, doc in enumerate(documents):
        document_token = nlp(doc)
        similarity = round(document_token.similarity(category), 2)
        similarities.append((f"Document {i+1}", similarity))
    return similarities
'''
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = lemmatize_text(text)
    return text

def remove_punctuation(text):
    punctuation = list(string.punctuation)
    return ''.join([char if char not in punctuation else ' ' for char in text])

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    return ' '.join([word for word in word_tokens if word.lower() not in stop_words])

def stem_text(text):
    word_tokens = word_tokenize(text)
    return ' '.join([stemmer.stem(word) for word in word_tokens])

def lemmatize_text(text):
    word_tokens = word_tokenize(text)
    return ' '.join([lemmatizer.lemmatize(word) for word in word_tokens])

def check_categorical_similarity(category, doc_type, doc):
    document = read_document(doc_type, doc)
    document = preprocess_text(document)
    category = preprocess_text(category)
    category_token = nlp(category)
    document_token = nlp(document)
    similarity = round(document_token.similarity(category_token), 2)
    return f"Similarity according to category: {similarity}"

def project_document_vectors(doc_text):
    sentences = [sent.text for sent in nlp(doc_text).sents]
    document_vectors = [nlp(sentence).vector for sentence in sentences]

    pca = PCA(n_components=4)
    transformed_vectors = pca.fit_transform(document_vectors)

    plt.figure(figsize=(10, 7))
    plt.scatter(transformed_vectors[:, 0], transformed_vectors[:, 1])

    for i, sentence in enumerate(sentences):
        plt.annotate(f"Sentence {i + 1}", xy=(transformed_vectors[i, 0], transformed_vectors[i, 1]))

    plt.title("Document Vectors PCA Projection")
    plt.grid(True)
    plt.show()
    plt.close()
    return plt

def pca_interface(doc_type, doc):
    doc_text = read_document(doc_type, doc)
    doc_text = preprocess_text(doc_text)
    plt = project_document_vectors(doc_text)
    return plt

def entity_recognize(doc_type, doc):
    document = read_document(doc_type, doc)
    document_token = nlp(document)
    entities = [(entity.text, entity.label_) for entity in document_token.ents]
    return entities

def display_entity_recognize(doc_type, doc):
    document = read_document(doc_type, doc)
    document_token = nlp(document)
    ner = displacy.render(document_token, style="ent", jupyter=True)
    return ner

def ner_interface(doc_type, doc):
    entities = entity_recognize(doc_type, doc)
    entity_display = display_entity_recognize(doc_type, doc)
    return entities, entity_display

def highlight_similar_words(pdf_1, pdf_2):
    document_1 = read_document("PDF", pdf_1.name)
    document_1 = preprocess_text(document_1)
    document_2 = read_document("PDF", pdf_2.name)
    document_2 = preprocess_text(document_2)

    doc_1_token = nlp(document_1)

    similar_words = set([token.text for token in doc_1_token if token.text in document_2])

    output_1 = pdf_1.name.replace(".pdf", "_highlighted.pdf")
    output_2 = pdf_2.name.replace(".pdf", "_highlighted.pdf")

    for pdf, output in zip([pdf_1.name, pdf_2.name], [output_1, output_2]):
        pdf_doc = pymupdf.open(pdf)
        for page_num in range(len(pdf_doc)):
            page = pdf_doc[page_num]
            for word in similar_words:
                rects = page.search_for(word)
                for rect in rects:
                    page.add_highlight_annot(rect)
        pdf_doc.save(output)

    return output_1, output_2

In [34]:
display_entity_recognize("TXT", "/content/txt1.txt")

In [35]:
#highlight_similar_words("/content/sample_doc_1.pdf", "/content/sample_doc_2.pdf", "out", "out2")

In [41]:
x = read_document("TXT", "/content/txt1.txt")

In [42]:
sentences = [sent.text for sent in nlp(x).sents]
print(sentences)

['Jackson is working at Microsoft.']


In [43]:
displacy.render(nlp(x), style="ent", jupyter=True)

In [39]:
read_document("PDF", "/content/document1.pdf")

'This is a sample text document.\nIt contains several lines of text.\nThe purpose is to compare this text with another document.\n'

In [44]:
entity_recognize("TXT", "/content/txt1.txt")

[('Jackson', 'PERSON'), ('Microsoft', 'ORG')]

In [30]:
 #project_document_vectors(["hello, world", "hello"])

In [31]:
similarity_demo = gr.Interface(
    fn=check_doc_similarity,
    inputs=[
        gr.Radio(["PDF", "TXT", "DOCX"], label="Document Type"),
        gr.File(label="Document 1"),
        gr.File(label="Document 2"),
    ],
    outputs=gr.Textbox(label="Similarity Score"),
    title="Document Similarity Detector",
    description="Upload two documents to check their similarity.",
)

categorical_similarity_demo = gr.Interface(
    fn=check_categorical_similarity,
    inputs=[
        gr.Textbox(label="Category"),
        gr.Radio(["PDF", "TXT", "DOCX"], label="Document Type"),
        gr.File(label="Document")
    ],
    outputs=gr.Textbox(label="Categorical Similarity Scores"),
    title="Categorical Similarity Checker",
    description="Check the similarity of documents to a given category."
)

highlight_similar_words_demo = gr.Interface(
    fn=highlight_similar_words,
    inputs=[
        gr.File(label="PDF 1"),
        gr.File(label="PDF 2")
    ],
    outputs=[
        gr.File(label="Highlighted PDF 1"),
        gr.File(label="Highlighted PDF 2")
    ],
    title="Highlight Similar Words in PDFs",
    description="Upload two PDF documents to highlight similar words between them."
)

ner_demo = gr.Interface(
    fn=ner_interface,
    inputs=[
        gr.Radio(["PDF", "TXT", "DOCX"], label="Document Type"),
        gr.File(label="Document")
    ],
    outputs=[
        gr.Textbox(label="Named Entities"),
        gr.Textbox(label="Entity Highlights"),
    ],
    title="Named Entity Recognition (NER)",
    description="Extract and display named entities from documents."
)

pca_demo = gr.Interface(
    fn=pca_interface,
    inputs=[
        gr.Radio(["PDF", "TXT", "DOCX"], label="Document Type"),
        gr.File(label="Document")
    ],
    outputs=gr.Plot(),
    title="Document PCA Visualization",
    description="Visualize document vectors using PCA."
)

demo = gr.TabbedInterface(
    interface_list=[
        similarity_demo,
        categorical_similarity_demo,
        highlight_similar_words_demo,
        ner_demo,
        pca_demo
    ],
    tab_names=[
        "Document Similarity",
        "Categorical Similarity",
        "Highlight Similar Words",
        "Named Entity Recognition",
        "PCA Visualization"
    ]
)

if __name__ == "__main__":
    demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://50691b4c27cf41745d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
